<a href="https://colab.research.google.com/github/Untick/InspectrumClinic_RS_gr1/blob/main/Marfida%20Alexander/Stajirovka_profpatolog_itogi_(A_Marfida).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **РЕКОМЕНДАТЕЛЬНАЯ СИСТЕМА ДЛЯ ПРОФПАТОЛОГА**
# **(INSPECTRUM CLINIC)**
# **<Стажировка>**


---


- Датасет DS1
  - 'https://storage.yandexcloud.net/terratraineeship/23_InspectrumClinic_RS/datasets/%D0%B4%D0%B0%D1%82%D0%B0%D1%81%D0%B5%D1%82%20%D0%BF%D1%80%D0%BE%D1%84%D0%BF%D0%B0%D1%82%D0%BE%D0%BB%D0%BE%D0%B3%20%D1%87%D0%B0%D1%81%D1%82%D1%8C.xlsx
- Датасет DS2
    - https://storage.yandexcloud.net/terratraineeship/23_InspectrumClinic_RS/datasets/26.06%20new%20dataset.zip
- Датасет DS3
    - https://storage.yandexcloud.net/terratraineeship/23_InspectrumClinic_RS/datasets/%D0%92%D0%B5%D1%81%D1%8C%20%D0%BF%D0%B5%D1%80%D0%B8%D0%BE%D0%B4.csv
  

# Загрузка библиотек. Монтирование Google Disk

In [ ]:
# Подключение библиотек и модулей
from google.colab import files # модуль для загрузки файлов в colab
import numpy as np # библиотека для работы с массивами данных
import pandas as pd # библиотека для обработки и анализа данных
import matplotlib.pyplot as plt # библиотека для визуализации данных
plt.style.use('dark_background')

### Для устранения предупреждения: Import "tensorflow.keras. ..." could not be resolved(reportMissingImports)
### вместо "tensorflow.keras" используется  "keras.api._v2.keras"

from keras.api._v2.keras import utils # модуль с утилитами кераса для преобразования категориальных данных в вектора
from keras.api._v2.keras.utils import plot_model
from keras.api._v2.keras.models import Sequential, Model # класс для создания последовательной модели нейронной сети
# Загружаем стандартные слои нейросети(полносвязный, регуляризации, нормализации, векторного представления слов, выравнивания, активации,...)
from keras.api._v2.keras.layers import Dense, Dropout, BatchNormalization, Embedding, Flatten, Activation, Input, SpatialDropout1D, concatenate
# Загружаем сверточные слои
from keras.api._v2.keras.layers import Conv1D, SpatialDropout1D, MaxPooling1D, GlobalAveragePooling1D, GlobalMaxPooling1D
# Загружаем рекуррентные слои
from keras.api._v2.keras.layers import LSTM, GRU, SimpleRNN, Bidirectional
# Загружаем алгоритмы обучения/оптимизации
from keras.api._v2.keras.optimizers import Adam, RMSprop, Adadelta, SGD, Adagrad

from keras.preprocessing.text import Tokenizer, text_to_word_sequence # инструменты предобработки текста от кераса
from sklearn.model_selection import train_test_split # модуль для разбиения данных на обучающую и тестовую выборки

# from sklearn import preprocessing # предварительная обработка данных
from sklearn.preprocessing import StandardScaler # масштабирование данных
# from sklearn.datasets import fetch_openml
# from sklearn.model_selection import cross_val_predict
# from sklearn.linear_model import SGDClassifier
from sklearn.metrics import mean_squared_error, mean_absolute_error # метрики для расчета ошибок
from sklearn.model_selection import train_test_split # разделение данных на выборки

# import gdown # загрузка датасетов из облака google
import re # регулярные выражения
import seaborn as sns # создание статистических графиков
sns.set_style('darkgrid')
from fastai.tabular import * # табличные данные
import os
# from PIL import Image
# Очистка экрана
from IPython.display import clear_output
# 'Магическая' команда python для отрисовки изображения в ноутбуке, а не в консоль или файл
%matplotlib inline

Монтирование Goodle-диска

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# ГЛОБАЛЬНЫЕ ПЕРЕМЕННЫЕ

In [ ]:
EMPTY_FIELDS = 'Не определено' # Замена NaN в датафреймах
DISK_DS1 = '/content/drive/MyDrive/УИИ(marphida)/Стажировка_Профпатолог/DS1'
DISK_DS2 = '/content/drive/MyDrive/УИИ(marphida)/Стажировка_Профпатолог/DS2'
DISK_DS3 = '/content/drive/MyDrive/УИИ(marphida)/Стажировка_Профпатолог/DS3'
ZMK_NM = 'ЗаключениеМК' # Наименование целевой колонки
ZMK_G = 'Годен' # Заключение МК "Годен"
ZMK_N = 'Не годен' # Заключение МК "Не годен"
ZMK_O = 'Ограниченно годен' # Заключение МК "Ограниченно годен"

# АНАЛИЗ И ПОДГОТОВКА DS1

## Загрузка и сохранение DS1

Чтение датасета из хранилища заказчика

In [ ]:
# gdown.download('https://storage.yandexcloud.net/terratraineeship/23_InspectrumClinic_RS/datasets/%D0%B4%D0%B0%D1%82%D0%B0%D1%81%D0%B5%D1%82%20%D0%BF%D1%80%D0%BE%D1%84%D0%BF%D0%B0%D1%82%D0%BE%D0%BB%D0%BE%D0%B3%20%D1%87%D0%B0%D1%81%D1%82%D1%8C.xlsx', None, quiet=True)

!wget -O data1.xlsx https://storage.yandexcloud.net/terratraineeship/23_InspectrumClinic_RS/datasets/%D0%B4%D0%B0%D1%82%D0%B0%D1%81%D0%B5%D1%82%20%D0%BF%D1%80%D0%BE%D1%84%D0%BF%D0%B0%D1%82%D0%BE%D0%BB%D0%BE%D0%B3%20%D1%87%D0%B0%D1%81%D1%82%D1%8C.xlsx

--2023-07-05 19:21:48--  https://storage.yandexcloud.net/terratraineeship/23_InspectrumClinic_RS/datasets/%D0%B4%D0%B0%D1%82%D0%B0%D1%81%D0%B5%D1%82%20%D0%BF%D1%80%D0%BE%D1%84%D0%BF%D0%B0%D1%82%D0%BE%D0%BB%D0%BE%D0%B3%20%D1%87%D0%B0%D1%81%D1%82%D1%8C.xlsx
Resolving storage.yandexcloud.net (storage.yandexcloud.net)... 213.180.193.243, 2a02:6b8::1d9
Connecting to storage.yandexcloud.net (storage.yandexcloud.net)|213.180.193.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 32352005 (31M) [application/vnd.openxmlformats-officedocument.spreadsheetml.sheet]
Saving to: ‘data1.xlsx’

data1.xlsx          100%[===================>]  30.85M  9.48MB/s    in 4.8s    

2023-07-05 19:21:54 (6.45 MB/s) - ‘data1.xlsx’ saved [32352005/32352005]



Сохранение в своём Google Disk

In [ ]:
!cp data1.xlsx '/content/drive/MyDrive/УИИ(marphida)/Стажировка_Профпатолог/DS1'

Чтение датасета с диска в датафрейм резервной копии



In [ ]:
# dfo = pd.read_excel('/content/drive/MyDrive/УИИ(marphida)/Стажировкийа_Профпатолог/ICdata_I(71).xlsx')
# df1_backup = pd.read_excel(DISK_DS1+'/data1.xlsx')
df1_backup = pd.read_excel(DISK_DS1+'/data1_without_err_1.xlsx')
# df_backup = pd.read_csv('/content/drive/MyDrive/УИИ(marphida)/Стажировка_Профпатолог/DS3/Весь_период(2023_06_03).csv',
                        # sep=';') #,
                        # index_col=[0]) #, ## index_col=False,
                        # names=['col_'+str(i) for i in range(1, neiro_shape[f_name][1]+1)])

Создание рабочего датафрейма

In [ ]:
df1 = df1_backup.copy()

## Обработка дубликатов строк/столбцов

### Обработка пустых столбцов

Удаление пустых столбцов

In [ ]:
del_empty_columns(df1)

Исходная размерность датафрейма: (1982, 4587)
Количество пустых столбцов: 884
Новая размерность датафрейма: (1982, 3703)


In [ ]:
df1.shape

(1982, 3703)

### Обработка дубликатов строк

In [ ]:
df1.shape

(1982, 3703)

Количество дубликатов строк

In [ ]:
# len(df1)-len(df1.drop_duplicates())
len(df1[df1.duplicated()])

56

Визуализация дубликатов строк

In [ ]:
df1[df1.duplicated()]

,ДокументПрохождениеМедосмотра,Клиент,КлиентДатаРождения,КлиентПол,КлиентПолПредставление,ДатаЗавершенияМедосмотра,ТипМедосмотра,ТипМедосмотраПредставление,ПсихОсвидетельствование,Контрагент,...,Оториноларингология3_Аудиометрия1_Значение,Оториноларингология3_Аудиометрия1_ЗначениеПредставление,Оториноларингология1_ЖалобыДополнение_Значение,Оториноларингология1_ЖалобыДополнение_ЗначениеПредставление,Стоматология2_ВП2_Значение,Стоматология2_ВП2_ЗначениеПредставление,Терапия2_ЗаключениеФОГК_Значение,Терапия2_ЗаключениеФОГК_ЗначениеПредставление,Офтальмология1_ДвигательныйАппаратГлазДополнение_Значение,Офтальмология1_ДвигательныйАппаратГлазДополнение_ЗначениеПредставление
392,2997d782-55ab-11ed-8108-0cc47aab8067,116fe6a5-455f-11ed-8107-0cc47aab8067,1977-03-06,Мужской,Мужской,2022-10-27,Периодический,Периодический,False,bd085484-1d16-11ed-8103-0cc47aab8067,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
395,61d271b9-55ab-11ed-8108-0cc47aab8067,c547414b-732c-11e6-8360-b8975a94e496,1963-01-17,Мужской,Мужской,2022-10-27,Периодический,Периодический,False,bd085484-1d16-11ed-8103-0cc47aab8067,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
396,fc62264f-55aa-11ed-8108-0cc47aab8067,116fe6a3-455f-11ed-8107-0cc47aab8067,1968-07-22,Мужской,Мужской,2022-10-27,Периодический,Периодический,False,bd085484-1d16-11ed-8103-0cc47aab8067,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
398,e334cb0e-55a6-11ed-8108-0cc47aab8067,116fe6a8-455f-11ed-8107-0cc47aab8067,1971-02-22,Женский,Женский,2022-10-27,Периодический,Периодический,False,bd085484-1d16-11ed-8103-0cc47aab8067,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
399,549479ee-55aa-11ed-8108-0cc47aab8067,24086b2f-4bf7-11e7-80d9-708bcd7f83cd,1976-06-16,Женский,Женский,2022-10-27,Периодический,Периодический,False,bd085484-1d16-11ed-8103-0cc47aab8067,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
410,832c79bc-55b5-11ed-8108-0cc47aab8067,39f65c6b-34f7-11e9-80ce-0cc47aab8067,1982-03-19,Мужской,Мужской,2022-10-27,Периодический,Периодический,False,963d16f4-d69e-11e7-80de-708bcd7f83cd,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
411,fb21176c-55bf-11ed-8108-0cc47aab8067,5ba8b3e2-81bf-11e9-80cf-0cc47aab8067,1969-09-02,Мужской,Мужской,2022-10-27,Периодический,Периодический,False,963d16f4-d69e-11e7-80de-708bcd7f83cd,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
429,30b084e0-5678-11ed-8108-0cc47aab8067,9fc8140c-bd93-11e8-80ce-0cc47aab8067,1970-01-21,Женский,Женский,2022-10-28,Периодический,Периодический,False,963d16f4-d69e-11e7-80de-708bcd7f83cd,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
430,00d62b6d-5678-11ed-8108-0cc47aab8067,dae940bb-c84c-11e8-80ce-0cc47aab8067,1970-09-06,Женский,Женский,2022-10-28,Периодический,Периодический,False,963d16f4-d69e-11e7-80de-708bcd7f83cd,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
431,723358f4-568f-11ed-8108-0cc47aab8067,5aafffb4-c344-11e9-80cf-0cc47aab8067,1979-03-04,Мужской,Мужской,2022-10-28,Периодический,Периодический,False,963d16f4-d69e-11e7-80de-708bcd7f83cd,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Удаление дубликатов строк

In [ ]:
df1 = df1.drop_duplicates()
df1.shape

(1926, 3703)

### Обработка пустых строк

In [ ]:
df1.shape

(1926, 3703)

In [ ]:
df1.dropna(axis=0, how='all', inplace=True)

In [ ]:
df1.shape

(1926, 3703)

Пустых строк не было

## Аназиз и обработка целевого столбца "ЗаключениеМК"

### Удаление записей без заключения

Пустые записи без заключения (NaN)



In [ ]:
df1[df1['ЗаключениеМК'].isna()].index

Int64Index([ 530,  531,  532,  533,  534,  535,  536,  563,  569,  580,
            ...
            1969, 1970, 1971, 1974, 1975, 1976, 1977, 1978, 1979, 1980],
           dtype='int64', length=407)

In [ ]:
df1[df1['ЗаключениеМК'].isna()]

,ДокументПрохождениеМедосмотра,Клиент,КлиентДатаРождения,КлиентПол,КлиентПолПредставление,ДатаЗавершенияМедосмотра,ТипМедосмотра,ТипМедосмотраПредставление,ПсихОсвидетельствование,Контрагент,...,Оториноларингология3_Аудиометрия1_Значение,Оториноларингология3_Аудиометрия1_ЗначениеПредставление,Оториноларингология1_ЖалобыДополнение_Значение,Оториноларингология1_ЖалобыДополнение_ЗначениеПредставление,Стоматология2_ВП2_Значение,Стоматология2_ВП2_ЗначениеПредставление,Терапия2_ЗаключениеФОГК_Значение,Терапия2_ЗаключениеФОГК_ЗначениеПредставление,Офтальмология1_ДвигательныйАппаратГлазДополнение_Значение,Офтальмология1_ДвигательныйАппаратГлазДополнение_ЗначениеПредставление
530,9a02747f-5991-11ed-8108-0cc47aab8067,e7c4834a-fb9f-11e9-80d2-0cc47aab8067,1968-09-01,Мужской,Мужской,2022-11-01,Периодический,Периодический,False,173d6ba7-12fe-11e7-80d3-708bcd7f83cd,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
531,3db750a6-59a1-11ed-8108-0cc47aab8067,a250f991-5a97-11e8-80ce-0cc47aab8067,1982-08-15,Мужской,Мужской,2022-11-01,Периодический,Периодический,False,173d6ba7-12fe-11e7-80d3-708bcd7f83cd,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
532,08361ad4-59a2-11ed-8108-0cc47aab8067,bf8e2211-f170-11e9-80d1-0cc47aab8067,1989-07-06,Мужской,Мужской,2022-11-01,Периодический,Периодический,False,173d6ba7-12fe-11e7-80d3-708bcd7f83cd,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
533,82b5c1da-59a2-11ed-8108-0cc47aab8067,fccc04ab-b984-11e7-80de-708bcd7f83cd,1985-06-25,Мужской,Мужской,2022-11-01,Периодический,Периодический,False,173d6ba7-12fe-11e7-80d3-708bcd7f83cd,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
534,e2586b19-59a4-11ed-8108-0cc47aab8067,9af0e1b3-1834-11e7-80d3-708bcd7f83cd,1965-01-22,Мужской,Мужской,2022-11-01,Периодический,Периодический,False,173d6ba7-12fe-11e7-80d3-708bcd7f83cd,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1976,467d8d4e-707d-11ed-8109-0cc47aab8067,01b3a76b-2dce-11e9-80ce-0cc47aab8067,1982-08-15,Мужской,Мужской,2022-11-30,Периодический,Периодический,False,ee33cfec-7da7-11ec-80fc-0cc47aab8067,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1977,137af572-70a5-11ed-8109-0cc47aab8067,41a7c665-7098-11ed-8109-0cc47aab8067,1972-11-12,Мужской,Мужской,2022-11-30,Периодический,Периодический,False,a9f70e4c-707f-11ed-8109-0cc47aab8067,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1978,26310d7e-70a5-11ed-8109-0cc47aab8067,41a7c663-7098-11ed-8109-0cc47aab8067,1986-07-09,Мужской,Мужской,2022-11-30,Периодический,Периодический,False,a9f70e4c-707f-11ed-8109-0cc47aab8067,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1979,b2602553-705c-11ed-8109-0cc47aab8067,0dc52156-f1a9-11e9-80d1-0cc47aab8067,1994-05-12,Мужской,Мужской,2022-11-30,Периодический,Периодический,False,e689a382-17b2-11e9-80ce-0cc47aab8067,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df1[df1['ЗаключениеМК'].isna()].index

Int64Index([ 530,  531,  532,  533,  534,  535,  536,  563,  569,  580,
            ...
            1969, 1970, 1971, 1974, 1975, 1976, 1977, 1978, 1979, 1980],
           dtype='int64', length=407)

In [ ]:
len(df1[df1['ЗаключениеМК'].isna()].index.to_list())

407

Удаление строк без данных в "ЗаключениеМК"

In [ ]:
df1.shape

(1926, 3703)

In [ ]:
df1.drop(df1[df1['ЗаключениеМК'].isna()].index, axis=0, inplace=True)

Размер датасета после удаления неопределённых записей целевого столбца:

In [ ]:
df1.shape

(1519, 3703)

In [ ]:
1926 - 1519

407

### Коррекция или удаление ошибочных записей

In [ ]:
prn_unique_txt_col(df1, 'ЗаключениеМК')

*** ЗаключениеМК(9 уникальных): ***
Годен
ГоденСКоррекциейЗрения
ГоденБезРаботНаВысотах
НуждаетсяВДообследованииИЛечении
ВременноНегоден
ОграниченноГоден
машинист бульдозера
Газоспасатель
 Газоспасатель



Удаление строк с ошибками

In [ ]:
values = [' Газоспасатель', 'Газоспасатель', 'машинист бульдозера']
print(df1.shape)
df1 = df1[df1['ЗаключениеМК'].isin(values) == False]
print(df1.shape)

(1519, 3703)
(1515, 3703)


Приводим целевой столбец с трафарету: 'Годен', 'Не годен', 'Ограниченно годен'

In [ ]:
change = {
    'ВременноНегоден':'Не годен',
    'ГоденБезРаботНаВысотах':'Годен',
    'ГоденСКоррекциейЗрения':'Годен',
    'НуждаетсяВДообследованииИЛечении':'Не годен',
    'ОграниченноГоден':'Ограниченно годен'
}
df1['ЗаключениеМК'] = df1['ЗаключениеМК'].replace(change)

<ipython-input-50-ce8b5e705573>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['ЗаключениеМК'] = df1['ЗаключениеМК'].replace(change)


In [ ]:
prn_unique_txt_col(df1, 'ЗаключениеМК', sort_asc = True)

*** ЗаключениеМК(3 уникальных): ***
Годен
Не годен
Ограниченно годен



In [ ]:
df1.shape

(1515, 3703)

### Сохранение предобработанного датасета

In [ ]:
df1.to_excel(DISK_DS1+'/data1_without_err_1.xlsx', index=False)
# df1 = pd.read_excel(DISK_DS1+'/data1_without_err_1.xlsx')

## Выявление наиболее информативных для обучения колонок

Начальная размерность датасета

In [ ]:
df1.shape

(1515, 3703)

In [ ]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1515 entries, 0 to 1514
Columns: 3703 entries, ДокументПрохождениеМедосмотра to Офтальмология1_ДвигательныйАппаратГлазДополнение_ЗначениеПредставление
dtypes: bool(1), datetime64[ns](43), float64(581), object(3078)
memory usage: 42.8+ MB


In [ ]:
df1.count()

ДокументПрохождениеМедосмотра                                             1515
Клиент                                                                    1515
КлиентДатаРождения                                                        1515
КлиентПол                                                                 1515
КлиентПолПредставление                                                    1515
                                                                          ... 
Стоматология2_ВП2_ЗначениеПредставление                                      1
Терапия2_ЗаключениеФОГК_Значение                                             1
Терапия2_ЗаключениеФОГК_ЗначениеПредставление                                1
Офтальмология1_ДвигательныйАппаратГлазДополнение_Значение                    1
Офтальмология1_ДвигательныйАппаратГлазДополнение_ЗначениеПредставление       1
Length: 3703, dtype: int64

### Разбор столбцов типа "float64"

In [ ]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1515 entries, 0 to 1514
Columns: 3703 entries, ДокументПрохождениеМедосмотра to Офтальмология1_ДвигательныйАппаратГлазДополнение_ЗначениеПредставление
dtypes: bool(1), datetime64[ns](43), float64(581), object(3078)
memory usage: 42.8+ MB


Анализ столбцов типа 'float64'

In [ ]:
 see_columns(df1, 'float64')

Выходные данные были обрезаны до нескольких последних строк (5000).
['Годен', 'Годен', 'Годен', 'Годен', 'Годен']
->96.0 = 2
[316, 929]
['Годен', 'Годен']
->105.0 = 2
[318, 668]
['Годен', 'Годен']
->73.0 = 6
[319, 525, 1169, 1240, 1324, 1399]
['Годен', 'Годен', 'Годен', 'Годен', 'Годен', 'Годен']
->108.0 = 5
[322, 365, 527, 1136, 1170]
['Годен', 'Годен', 'Годен', 'Годен', 'Годен']
->63.0 = 2
[328, 928]
['Годен', 'Годен']
->101.0 = 5
[346, 442, 466, 501, 1391]
['Годен', 'Годен', 'Годен', 'Годен', 'Годен']
->93.0 = 2
[347, 653]
['Годен', 'Годен']
->79.0 = 13
[356, 422, 424, 834, 864, 867, 980, 993, 1031, 1132, 1180, 1397, 1475]
['Годен', 'Годен', 'Годен', 'Годен', 'Годен', 'Годен', 'Годен', 'Годен', 'Годен', 'Годен', 'Годен', 'Годен', 'Годен']
->67.0 = 5
[361, 478, 839, 1252, 1457]
['Годен', 'Годен', 'Годен', 'Годен', 'Годен']
->107.0 = 4
[366, 403, 421, 463]
['Годен', 'Годен', 'Годен', 'Годен']
->106.0 = 2
[395, 528]
['Годен', 'Годен']
->94.0 = 4
[404, 447, 526, 1135]
['Годен', 'Годен',

Результаты разбора поностью не приводятся ввиду объёмности.
- Качественная картина:
  - заполненность и уникальность записей варьируются от 0 до 1515
  - записи с 1 уникальным значением не имеют потенциала обучения, поскольку, даже при большой заполненности не несут новой информации. Как правило, это формальные повторяющиеся значения, типичные для стандартного нормального случая.

Для обучающего датасета оставляем столбцы:
- с заполненностью не менее 50 записей
- из них с уникальными не менее 2 записей

In [ ]:
df1.shape

(1515, 3170)

### Разбор столбцов типа "object"

In [ ]:
see_columns(columns_type)


col№:1 *** ЛабораторныеИсследования1_ХР_Рекомендация1: ***
Количество непустых элементов:  0
Количество уникальных элементов:  0 
----------------------------------
....уникальная запись.... = число повторов
[индексы строк с записями...]
col№:20 *** Оториноларингология1_Классификация11: ***
Количество непустых элементов:  0
Количество уникальных элементов:  0 
----------------------------------
....уникальная запись.... = число повторов
[индексы строк с записями...]
col№:300 *** Терапия2_АртериальноеДавление21_Значение: ***
Количество непустых элементов:  106
Количество уникальных элементов:  30 
----------------------------------
....уникальная запись.... = число повторов
[индексы строк с записями...]
->90.0 = 7
[28, 473, 481, 1007, 1009, 1337, 1484]
->70.0 = 14
[29, 367, 477, 670, 871, 903, 904, 906, 907, 908, 911, 913, 914, 1140]
->80.0 = 35
[31, 56, 135, 137, 147, 303, 375, 391, 467, 475, 480, 546, 547, 644, 787, 831, 893, 910, 999, 1000, 1001, 1002, 1003, 1004, 1005, 1006, 1008, 1

Результаты разбора поностью не приводятся ввиду объёмности.
- Качественная картина:
  - заполненность и уникальность записей варьируются от 0 до 1515
  - записи с 1 уникальным значением не имеют потенциала обучения, поскольку, даже при большой заполненности не несут новой информации. Как правило, это формальные повторяющиеся значения, типичные для стандартного нормального случая.

In [ ]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1515 entries, 0 to 1514
Columns: 3703 entries, ДокументПрохождениеМедосмотра to Офтальмология1_ДвигательныйАппаратГлазДополнение_ЗначениеПредставление
dtypes: bool(1), datetime64[ns](43), float64(581), object(3078)
memory usage: 42.8+ MB


Анализ столбцов типа 'object'

In [ ]:
dff = pd.DataFrame(columns=['col', 'fill', 'uniq', 'col_ind'])
# dfo = df1.select_dtypes(include=['object'])
dfo = df1.select_dtypes(include=['float64'])
for colname in dfo.columns:
  dff.loc[len(dff.index)] = [
      colname,
      len(dfo[~dfo[colname].isna()]),
      len(dfo.loc[~dfo[colname].isnull(), colname].unique()),
      df1.columns.get_loc(colname)
  ]
#dff.sort_values(
#        by = ["fill", "uniq"],
#        ascending = False)
print(len(dff))

581


Для обучающего датасета оставляем столбцы:
- с заполненностью не менее 50 записей
- из них с уникальностью не менее 2 записей

In [ ]:
shrinking_columns_quantity(df1, 'object', 50, 2):

Количество столбцов с заполненностью > 50 = 127
Из предыдущих столбцов количество с уникальными элементами > 2 (оставляемых столбцов) = 48
Характеристика оставляемых столбцов:
Заполненность: min=62; max=1328
Уникальные:min=3; max=633



Из числовых столбцов оставлено для обучающей выборки:

In [ ]:
len(dff_fill_50_plus[dff_fill_50_plus['uniq']>2])

48

In [ ]:
df1.shape

(1515, 3170)

Удаляем исключаемые из обучения столбцы

Логически вычитаем выбранные 48 столбцов из индексов всех столбцов типа "float64"

In [ ]:
len(dff['col_ind'].index.difference(dff_f_50_u_2['col_ind'].index))

533

In [ ]:
533 + 48

581

In [ ]:
len(dff_f_50_u_2['col_ind'])

48

Удаляем из датасета DS1 все "object" без оставляемых 48 для обучающей выборки

In [ ]:
# dff['col_ind'].index.difference(dff_f_50_u_2['col_ind'].index)
# df1.columns[dff_f_50_u_2['col_ind']]
df1.drop(df1.columns[dff['col_ind'].index.difference(dff_f_50_u_2['col_ind'].index)],
         axis= 1,
         inplace=True)

In [ ]:
dff['col_ind'].index.difference(dff_f_50_u_2['col_ind'].index)

Int64Index([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
            ...
            571, 572, 573, 574, 575, 576, 577, 578, 579, 580],
           dtype='int64', length=533)

In [ ]:
dff_f_50_u_2

,col,fill,uniq,col_ind
65,Терапия1_П_Значение,1328,66,497
66,Терапия1_П_ЗначениеПредставление,1328,66,498
122,Терапия1_Рост_Значение,1281,54,717
123,Терапия1_Рост_ЗначениеПредставление,1284,54,718
124,Терапия1_Вес_ЗначениеПредставление,1282,89,720
125,УЗИДиагностика1_Головка_Значение,76,15,771
126,УЗИДиагностика1_Головка_ЗначениеПредставление,76,15,772
127,УЗИДиагностика1_Тело_Значение,83,13,773
128,УЗИДиагностика1_Тело_ЗначениеПредставление,83,13,774
165,Терапия2_П_Значение,105,35,936


In [ ]:
df1.columns[dff_f_50_u_2['col_ind']]

Index(['Терапия1_П_Значение', 'Терапия1_П_ЗначениеПредставление',
       'Терапия1_Рост_Значение', 'Терапия1_Рост_ЗначениеПредставление',
       'Терапия1_Вес_ЗначениеПредставление',
       'УЗИДиагностика1_Головка_Значение',
       'УЗИДиагностика1_Головка_ЗначениеПредставление',
       'УЗИДиагностика1_Тело_Значение',
       'УЗИДиагностика1_Тело_ЗначениеПредставление', 'Терапия2_П_Значение',
       'Терапия2_П_ЗначениеПредставление', 'Терапия2_Рост_Значение',
       'Терапия2_Рост_ЗначениеПредставление', 'Терапия2_Вес_Значение',
       'Терапия2_Вес_ЗначениеПредставление',
       'УЗИДиагностика1_РазмерыДлина_Значение',
       'УЗИДиагностика1_РазмерыДлина_ЗначениеПредставление',
       'УЗИДиагностика1_ПереднеЗадний_Значение',
       'УЗИДиагностика1_ПереднеЗадний_ЗначениеПредставление',
       'УЗИДиагностика1_ТолщинаЛевыйЯичник_Значение',
       'УЗИДиагностика1_ТолщинаЛевыйЯичник_ЗначениеПредставление',
       'УЗИДиагностика1_ДлинаЛевыйЯичник_Значение',
       'УЗИДиагностика1_

## Заменяем NaN на EMPTY_FIELDS ('Не определено')

In [ ]:
df1 = nan_to_neo(df1)

Всего в датафрейме количество NaN =  8106978
Заменяем NaN на "Не определено"
Всего в датафрейме осталось количество NaN =  0


In [ ]:
df1.head(10).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Columns: 4587 entries, ДокументПрохождениеМедосмотра to Офтальмология1_ДвигательныйАппаратГлазДополнение_ЗначениеПредставление
dtypes: bool(1), datetime64[ns](43), float64(1253), object(3290)
memory usage: 358.4+ KB


In [ ]:
#df1.loc[df1.dtypes=='float64']
#df1[df1.dtypes=='datetime64[ns]']
df1[df1['Клиент']=='7fe324e0-fa51-11eb-80f6-0cc47aab8067']

,ДокументПрохождениеМедосмотра,Клиент,КлиентДатаРождения,КлиентПол,КлиентПолПредставление,ДатаЗавершенияМедосмотра,ТипМедосмотра,ТипМедосмотраПредставление,ПсихОсвидетельствование,Контрагент,...,Оториноларингология3_Аудиометрия1_Значение,Оториноларингология3_Аудиометрия1_ЗначениеПредставление,Оториноларингология1_ЖалобыДополнение_Значение,Оториноларингология1_ЖалобыДополнение_ЗначениеПредставление,Стоматология2_ВП2_Значение,Стоматология2_ВП2_ЗначениеПредставление,Терапия2_ЗаключениеФОГК_Значение,Терапия2_ЗаключениеФОГК_ЗначениеПредставление,Офтальмология1_ДвигательныйАппаратГлазДополнение_Значение,Офтальмология1_ДвигательныйАппаратГлазДополнение_ЗначениеПредставление
2,9cbb191e-8e24-11ec-80fe-0cc47aab8067,7fe324e0-fa51-11eb-80f6-0cc47aab8067,1985-09-08,Мужской,Мужской,2022-02-15,Периодический,Периодический,False,a6d559b0-7835-11eb-80e8-0cc47aab8067,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#df1.drop_duplicates()
df1d = df1[df1.duplicated()].copy()
#df1.duplicated()
df1d

,ДокументПрохождениеМедосмотра,Клиент,КлиентДатаРождения,КлиентПол,КлиентПолПредставление,ДатаЗавершенияМедосмотра,ТипМедосмотра,ТипМедосмотраПредставление,ПсихОсвидетельствование,Контрагент,...,Оториноларингология3_Аудиометрия1_Значение,Оториноларингология3_Аудиометрия1_ЗначениеПредставление,Оториноларингология1_ЖалобыДополнение_Значение,Оториноларингология1_ЖалобыДополнение_ЗначениеПредставление,Стоматология2_ВП2_Значение,Стоматология2_ВП2_ЗначениеПредставление,Терапия2_ЗаключениеФОГК_Значение,Терапия2_ЗаключениеФОГК_ЗначениеПредставление,Офтальмология1_ДвигательныйАппаратГлазДополнение_Значение,Офтальмология1_ДвигательныйАппаратГлазДополнение_ЗначениеПредставление
392,2997d782-55ab-11ed-8108-0cc47aab8067,116fe6a5-455f-11ed-8107-0cc47aab8067,1977-03-06,Мужской,Мужской,2022-10-27,Периодический,Периодический,False,bd085484-1d16-11ed-8103-0cc47aab8067,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
395,61d271b9-55ab-11ed-8108-0cc47aab8067,c547414b-732c-11e6-8360-b8975a94e496,1963-01-17,Мужской,Мужской,2022-10-27,Периодический,Периодический,False,bd085484-1d16-11ed-8103-0cc47aab8067,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
396,fc62264f-55aa-11ed-8108-0cc47aab8067,116fe6a3-455f-11ed-8107-0cc47aab8067,1968-07-22,Мужской,Мужской,2022-10-27,Периодический,Периодический,False,bd085484-1d16-11ed-8103-0cc47aab8067,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
398,e334cb0e-55a6-11ed-8108-0cc47aab8067,116fe6a8-455f-11ed-8107-0cc47aab8067,1971-02-22,Женский,Женский,2022-10-27,Периодический,Периодический,False,bd085484-1d16-11ed-8103-0cc47aab8067,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
399,549479ee-55aa-11ed-8108-0cc47aab8067,24086b2f-4bf7-11e7-80d9-708bcd7f83cd,1976-06-16,Женский,Женский,2022-10-27,Периодический,Периодический,False,bd085484-1d16-11ed-8103-0cc47aab8067,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
410,832c79bc-55b5-11ed-8108-0cc47aab8067,39f65c6b-34f7-11e9-80ce-0cc47aab8067,1982-03-19,Мужской,Мужской,2022-10-27,Периодический,Периодический,False,963d16f4-d69e-11e7-80de-708bcd7f83cd,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
411,fb21176c-55bf-11ed-8108-0cc47aab8067,5ba8b3e2-81bf-11e9-80cf-0cc47aab8067,1969-09-02,Мужской,Мужской,2022-10-27,Периодический,Периодический,False,963d16f4-d69e-11e7-80de-708bcd7f83cd,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
429,30b084e0-5678-11ed-8108-0cc47aab8067,9fc8140c-bd93-11e8-80ce-0cc47aab8067,1970-01-21,Женский,Женский,2022-10-28,Периодический,Периодический,False,963d16f4-d69e-11e7-80de-708bcd7f83cd,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
430,00d62b6d-5678-11ed-8108-0cc47aab8067,dae940bb-c84c-11e8-80ce-0cc47aab8067,1970-09-06,Женский,Женский,2022-10-28,Периодический,Периодический,False,963d16f4-d69e-11e7-80de-708bcd7f83cd,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
431,723358f4-568f-11ed-8108-0cc47aab8067,5aafffb4-c344-11e9-80cf-0cc47aab8067,1979-03-04,Мужской,Мужской,2022-10-28,Периодический,Периодический,False,963d16f4-d69e-11e7-80de-708bcd7f83cd,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Список дубликатов строк
df1[df1.duplicated()]

,ДокументПрохождениеМедосмотра,Клиент,КлиентДатаРождения,КлиентПол,КлиентПолПредставление,ДатаЗавершенияМедосмотра,ТипМедосмотра,ТипМедосмотраПредставление,ПсихОсвидетельствование,Контрагент,...,Оториноларингология3_Аудиометрия1_Значение,Оториноларингология3_Аудиометрия1_ЗначениеПредставление,Оториноларингология1_ЖалобыДополнение_Значение,Оториноларингология1_ЖалобыДополнение_ЗначениеПредставление,Стоматология2_ВП2_Значение,Стоматология2_ВП2_ЗначениеПредставление,Терапия2_ЗаключениеФОГК_Значение,Терапия2_ЗаключениеФОГК_ЗначениеПредставление,Офтальмология1_ДвигательныйАппаратГлазДополнение_Значение,Офтальмология1_ДвигательныйАппаратГлазДополнение_ЗначениеПредставление
392,2997d782-55ab-11ed-8108-0cc47aab8067,116fe6a5-455f-11ed-8107-0cc47aab8067,1977-03-06,Мужской,Мужской,2022-10-27,Периодический,Периодический,False,bd085484-1d16-11ed-8103-0cc47aab8067,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
395,61d271b9-55ab-11ed-8108-0cc47aab8067,c547414b-732c-11e6-8360-b8975a94e496,1963-01-17,Мужской,Мужской,2022-10-27,Периодический,Периодический,False,bd085484-1d16-11ed-8103-0cc47aab8067,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
396,fc62264f-55aa-11ed-8108-0cc47aab8067,116fe6a3-455f-11ed-8107-0cc47aab8067,1968-07-22,Мужской,Мужской,2022-10-27,Периодический,Периодический,False,bd085484-1d16-11ed-8103-0cc47aab8067,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
398,e334cb0e-55a6-11ed-8108-0cc47aab8067,116fe6a8-455f-11ed-8107-0cc47aab8067,1971-02-22,Женский,Женский,2022-10-27,Периодический,Периодический,False,bd085484-1d16-11ed-8103-0cc47aab8067,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
399,549479ee-55aa-11ed-8108-0cc47aab8067,24086b2f-4bf7-11e7-80d9-708bcd7f83cd,1976-06-16,Женский,Женский,2022-10-27,Периодический,Периодический,False,bd085484-1d16-11ed-8103-0cc47aab8067,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
410,832c79bc-55b5-11ed-8108-0cc47aab8067,39f65c6b-34f7-11e9-80ce-0cc47aab8067,1982-03-19,Мужской,Мужской,2022-10-27,Периодический,Периодический,False,963d16f4-d69e-11e7-80de-708bcd7f83cd,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
411,fb21176c-55bf-11ed-8108-0cc47aab8067,5ba8b3e2-81bf-11e9-80cf-0cc47aab8067,1969-09-02,Мужской,Мужской,2022-10-27,Периодический,Периодический,False,963d16f4-d69e-11e7-80de-708bcd7f83cd,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
429,30b084e0-5678-11ed-8108-0cc47aab8067,9fc8140c-bd93-11e8-80ce-0cc47aab8067,1970-01-21,Женский,Женский,2022-10-28,Периодический,Периодический,False,963d16f4-d69e-11e7-80de-708bcd7f83cd,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
430,00d62b6d-5678-11ed-8108-0cc47aab8067,dae940bb-c84c-11e8-80ce-0cc47aab8067,1970-09-06,Женский,Женский,2022-10-28,Периодический,Периодический,False,963d16f4-d69e-11e7-80de-708bcd7f83cd,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
431,723358f4-568f-11ed-8108-0cc47aab8067,5aafffb4-c344-11e9-80cf-0cc47aab8067,1979-03-04,Мужской,Мужской,2022-10-28,Периодический,Периодический,False,963d16f4-d69e-11e7-80de-708bcd7f83cd,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# АНАЛИЗ И ПОДГОТОВКА DS2

## Титульная информация датасета DS2

In [ ]:
# Словарь датафреймов датасета DS2
neiro_name = {
  'neiro_01_22': pd.DataFrame(), # + neiro_01-22
  'neiro_02_22': pd.DataFrame(), # + neiro_02-22
  'neiro_03_22': pd.DataFrame(), # + neiro_03-22
  'neiro_04_22': pd.DataFrame(), # + neiro_04-22
  'neiro_05_22': pd.DataFrame(), # + neiro_05-22
  'neiro_06_22': pd.DataFrame(), # + neiro_06-22
  'neiro_07_22': pd.DataFrame(), # + neiro_07-22
  'neiro_08_22': pd.DataFrame(), # + neiro-08-22
  'neiro_09_22': pd.DataFrame(), # (1) + neiro-09-22
  'neiro_10_22': pd.DataFrame(), # ---!!! neiro-10-22
  'neiro_12_22': pd.DataFrame(), # + neiro_12-22
  'neiro_01_23': pd.DataFrame(), # + neiro_01_23
  'neiro_02_23': pd.DataFrame(), # + neiro_02_23
  'neiro_03_23': pd.DataFrame(), # + neiro_03_23
  'neiro_04_23': pd.DataFrame(), # + neiro_04_23
  'neiro_05_23': pd.DataFrame(), # + neiro_05_23
  'neiro_06_23': pd.DataFrame()  # + neiro_06-23
}
# Размерности файлов датасета DS2
neiro_shape = {
  'neiro_01_22': (1511, 342),
  'neiro_02_22': (1433, 294),
  'neiro_03_22': (2116, 446),
  'neiro_04_22': (1849, 306),
  'neiro_05_22': (1831, 262),
  'neiro_06_22': (1890, 458),
  'neiro_07_22': (1069, 218),
  'neiro_08_22': (1727, 304),
  'neiro_09_22': (1746, 310),
  'neiro_10_22': (0, 0),
  'neiro_12_22': (1235, 222),
  'neiro_01_23': (1317, 228),
  'neiro_02_23': (1362, 290),
  'neiro_03_23': (1501, 430),
  'neiro_04_23': (1420, 266),
  'neiro_05_23': (2532, 296),
  'neiro_06_23': (1041, 189)
}

# neiro_name = pd.DataFrame(cols)

In [ ]:
for nm in neiro_name:
  print(f'{nm}.shape = {neiro_name[nm].shape}')

neiro_01_22.shape = (1512, 342)
neiro_02_22.shape = (1434, 294)
neiro_03_22.shape = (2117, 446)
neiro_04_22.shape = (1850, 306)
neiro_05_22.shape = (1832, 262)
neiro_06_22.shape = (1891, 458)
neiro_07_22.shape = (1070, 218)
neiro_08_22.shape = (1728, 304)
neiro_09_22.shape = (1747, 310)
neiro_10_22.shape = (0, 0)
neiro_12_22.shape = (1236, 222)
neiro_01_23.shape = (1318, 228)
neiro_02_23.shape = (1363, 290)
neiro_03_23.shape = (1502, 430)
neiro_04_23.shape = (1421, 266)
neiro_05_23.shape = (2533, 296)
neiro_06_23.shape = (1041, 189)


## DS2: Преобразование CSV в EXEL. Сохранение на диске

In [ ]:
# Перебор папок в директории базы
dir_ndf = '/content/drive/MyDrive/УИИ(marphida)/Стажировка_Профпатолог/DS2'

iter_neiro = list(iter(neiro_name))

for patch in os.listdir(dir_ndf):
  #  if patch[:-4] not in cols:
  if patch[:-4] not in iter_neiro:
    continue
  if patch[:-4] == 'neiro_10_22':  ## Не читается из-за ошибки
    continue
  print('Старт "', patch[:-4], '"', sep='')
#  patch[:-4] = pd.read_csv(dir_ndf+'/'+patch, sep=';')
  f_name = patch[:-4]
  if f_name == 'neiro_06_23':
    csv_tmp = pd.read_csv(dir_ndf+'/'+patch, sep=';', index_col=[0]) ## index_col=False)
  else:
    csv_tmp = pd.read_csv(dir_ndf+'/'+patch, sep=';', index_col=[0], ## index_col=False,
                          names=['col_'+str(i) for i in range(1, neiro_shape[f_name][1]+1)])

  csv_tmp.to_excel(dir_ndf+'/'+f_name+'.xlsx', index=False)
  print('Финиш "', patch[:-4], '"', sep='')
#  print(dir_ndf+'/'+patch)



Старт "neiro_05_23"


<ipython-input-51-8163fee7043c>:18: DtypeWarning: Columns (54,80,81,82,83,84,85,98,99,100,101,102,103,106,107,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295) have mixed types. Specify dtype option on import or set low_memory=False.
  csv_tmp = pd.read_csv(dir_ndf+'/'+patch, sep=';', index_col=[0], ## index_col=False,


Финиш "neiro_05_23"
Старт "neiro_02_22"
Финиш "neiro_02_22"
Старт "neiro_04_23"
Финиш "neiro_04_23"
Старт "neiro_03_22"


<ipython-input-51-8163fee7043c>:18: DtypeWarning: Columns (10,11,12,13,14,22,23,24,25,26,27,32,33,34,35,36,37,38,39,40,41,42,43,50,51,52,53,54,55,80,81,82,83,84,85,88,89,96,97,98,99,108,109,110,111,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,166,167,168,169,170,171,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,204,205,206,207,208,209,214,215,218,219,220,221,222,223,224,225,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,35

Финиш "neiro_03_22"
Старт "neiro_05_22"
Финиш "neiro_05_22"
Старт "neiro_06_22"
Финиш "neiro_06_22"
Старт "neiro_02_23"
Финиш "neiro_02_23"
Старт "neiro_12_22"
Финиш "neiro_12_22"
Старт "neiro_06_23"
Финиш "neiro_06_23"
Старт "neiro_03_23"
Финиш "neiro_03_23"
Старт "neiro_01_22"
Финиш "neiro_01_22"
Старт "neiro_07_22"
Финиш "neiro_07_22"
Старт "neiro_08_22"
Финиш "neiro_08_22"
Старт "neiro_04_22"
Финиш "neiro_04_22"
Старт "neiro_01_23"
Финиш "neiro_01_23"
Старт "neiro_09_22"
Финиш "neiro_09_22"


## DS2: загрузка файлов датасета

In [ ]:
dir_ndf = '/content/drive/MyDrive/УИИ(marphida)/Стажировка_Профпатолог/DS2'
for patch in os.listdir(dir_ndf):
  is_xlsx = patch[:-4]+'.xlsx'
  if is_xlsx in os.listdir(dir_ndf):
    print(f'Загрузка "{is_xlsx}"')
    neiro_name[patch[:-4]] = pd.read_excel(dir_ndf+'/'+is_xlsx)

Загрузка "neiro_05_23.xlsx"
Загрузка "neiro_02_22.xlsx"
Загрузка "neiro_04_23.xlsx"
Загрузка "neiro_03_22.xlsx"
Загрузка "neiro_05_22.xlsx"
Загрузка "neiro_06_22.xlsx"
Загрузка "neiro_02_23.xlsx"
Загрузка "neiro_12_22.xlsx"
Загрузка "neiro_06_23.xlsx"
Загрузка "neiro_03_23.xlsx"
Загрузка "neiro_01_22.xlsx"
Загрузка "neiro_07_22.xlsx"
Загрузка "neiro_08_22.xlsx"
Загрузка "neiro_04_22.xlsx"
Загрузка "neiro_01_23.xlsx"
Загрузка "neiro_09_22.xlsx"


In [ ]:
for nm in neiro_name:
  print(f'{nm}')
  print(neiro_name[nm].iloc[:3, :3], '...', neiro_name[nm].iloc[:3, :-3])
  break

neiro_01_22
                                  col_2                col_3    col_4
0  dca98d4e-2098-11ea-80d6-0cc47aab8067  1989-12-31T00:00:00  Женский
1  b35fb2bb-9c35-11eb-80eb-0cc47aab8067  1998-07-20T00:00:00  Женский
2  f8c98e39-9c2c-11eb-80eb-0cc47aab8067  2002-01-04T00:00:00  Женский ...                                   col_2                col_3    col_4  \
0  dca98d4e-2098-11ea-80d6-0cc47aab8067  1989-12-31T00:00:00  Женский   
1  b35fb2bb-9c35-11eb-80eb-0cc47aab8067  1998-07-20T00:00:00  Женский   
2  f8c98e39-9c2c-11eb-80eb-0cc47aab8067  2002-01-04T00:00:00  Женский   

     col_5                col_6          col_7          col_8  col_9  \
0  Женский  2021-02-02T00:00:00  Периодический  Периодический  False   
1  Женский  2021-04-13T00:00:00  Периодический  Периодический  False   
2  Женский  2021-04-13T00:00:00  Периодический  Периодический  False   

                                 col_10 col_11  ... col_331 col_332 col_333  \
0  3d01b2d6-ae08-11e9-80cf-0cc47aab8067    

# АНАЛИЗ И ПОДГОТОВКА DS3




---



# СЛУЖЕБНЫЕ ФУНКЦИИ

## Замена всех NaN на EMPTY_FIELDS ("Не определено")

In [ ]:
def nan_to_neo(df):
  # df.describe(include = "all")
  print('Всего в датафрейме количество NaN =', df.isna().sum().sum())
  print(f'Заменяем NaN на "{EMPTY_FIELDS}"')
  df = df.fillna(EMPTY_FIELDS)
  print('Всего в датафрейме осталось количество NaN =', df.isna().sum().sum())
  return df

## Подсчёт заполненности колонок

In [ ]:
# Аргументы - датафрейм и список наименований столбцов
# Исключения: NaN
#
def count_columns_fill_not_na(df, arg_list):
  count_list = 0
  print('Текущая структура датафрейма: ', df.shape, '\n')
  for col_name in arg_list:
    count_list += 1
    print(f'*** {col_name}: ***')
    print('Количество непустых элементов: ', len(df[~df[col_name].isna()]) )
    print('Количество уникальных элементов: ', len(df.loc[~df[col_name].isnull(), col_name].unique()), '\n')
  print('Количество столбцов: ', count_list)

In [ ]:
# Аргументы - датафрейм и список наименований столбцов
# Исключения: EMPTY_FIELDS ("Не определено")
#
def count_columns_fill(df, arg_list):
  count_list = 0
  print('Текущая структура датафрейма: ', df.shape, '\n')
  for col_name in arg_list:
    count_list += 1
    print(f'*** {col_name}: ***')
    print('Количество непустых элементов: ', len(df[df[col_name] != EMPTY_FIELDS]) )
    print('Количество уникальных элементов: ', len(df.loc[df[col_name] != EMPTY_FIELDS, col_name].unique()), '\n')
  print('Количество столбцов: ', count_list)

## Подсчёт заполненности столбцов

In [ ]:
# Аргументы - параметры
def count_columns_fill(df, *arg):
  print('Текущая структура датафрейма: ', df.shape, '\n')
  for colname in arg:
    print(f'*** {colname}: ***')
    print('Количество непустых элементов: ', len(df[~df[colname].isna()]))
    print('Количество уникальных элементов: ', len(df.loc[~df[colname].isnull(), colname].unique()), '\n')

In [ ]:
# Аргументы - список
def count_columns_in_list_fill(df, arg_list):
  count_list = 0
  print('Текущая структура датафрейма: ', df.shape, '\n')
  for colname in arg_list:
    count_list += 1
    print(f'*** {colname}: ***')
    print('Количество непустых элементов: ', len(df[~df[colname].isna()]) )
    print('Количество уникальных элементов: ', len(df.loc[~df[colname].isnull(), colname].unique()), '\n')
  print('Количество столбцов: ', count_list)

## Получение количества значащих записей и уникальных элементов в столбце

In [ ]:
# Аргументы - датафрейм и строка наименования столбца
# return: count_fills, count_unique
#
def count_fill_and_unique(df, col_name):
  return len(df[df[col_name] != EMPTY_FIELDS]), len(df.loc[df[col_name] != EMPTY_FIELDS, col_name].unique())


## Просмотр содержания ненулевых уникальных значений полей в выбранных текстовых столбцах

In [ ]:
def prn_unique_txt_col(df, *arg, sort_asc = False):
  for colname in arg:
    column_list = list(df.loc[~df[colname].isnull(), colname].unique())
    count_list = len(column_list)
    print(f'*** {colname}({count_list} уникальных): ***')
    to_join = sorted(column_list) if sort_asc else column_list
    print('\n'.join(to_join))
    print()

## Удаление из списка малоинформативных столбцов



In [ ]:
def reduce_colomns_unique_1(df, dfcols_to_save):
  ret = []
  print('Входящий размер списка столбцов: ', len(dfcols_to_save))
  for colname in dfcols_to_save:
    if len(df.loc[~df[colname].isnull(), colname].unique()) != 1:
      ret.append(colname)
  print('Исходящий размер списка столбцов: ', len(ret))
  ret
  return ret

## Сравнение пары списков на совпадение

In [ ]:
def is_equal_lst_cols(col1, col2):
  res = [x for x in col1 + col2 if x not in col1 or x not in col2]
  if not res:
    print("Списки совпаают")
  else:
    print("*** Несовпадающие элементы списков: ***")
    print('\n'.join(res))

## Сравнение пары колонок датафрейма на совпадение

In [ ]:
def is_equal_df_cols(df, col1, col2):
  dfcmp = np.where(df[col1] == df[col2], True, False)
  df_not_equal_indexes = df[~dfcmp].index
  print(df[~dfcmp][[col1, col2]])
#  return df_not_equal_indexes

## Проверка наличия пустых столбцов и их удаление:

In [ ]:
def del_empty_columns(df):
#  dn = df.isnull().sum()
  print('Исходная размерность датафрейма:', df.shape)
#  print('Количество пустых столбцов:', len(dn[dn == 1926]))
  print('Количество пустых столбцов:', len(df.loc[:, df.isna().all()].columns))
  df.dropna(axis=1, how='all', inplace=True)
  print('Новая размерность датафрейма:', df.shape)



---



Получение Serias с пустыми столбцами

In [ ]:
# epmty_columns_DS1 = df1.loc[:, df1.isna().all()].columns

Сохранение списка пустых столбцов

In [ ]:
#epmty_columns_DS1.values
# np.save(DISK_DS1+'/epmty_columns_DS1.lst', epmty_columns_DS1.values)
# e_c_DS1 = np.load(DISK_DS1+'/epmty_columns_DS1.lst.npy', allow_pickle=True)

In [ ]:
# e_c_DS1



---



## Сравнение строк датасета(ов). Возврат количества совпавших и несовпавших значений

In [ ]:
# Возврат: количество True, количество False
def cmp_raws(df1, raw1, df2, raw2):
  tr = (df1.iloc[raw1,:]==df2.iloc[raw2,:])[:].to_list().count(True)
  fl = (df1.iloc[raw1,:]==df2.iloc[raw2,:])[:].to_list().count(False)
  print(f'Совпало: {tr}; Не совпало: {fl}')
  return tr, fl

## Поиск совпадающих строк в датасете(ах)

In [ ]:
def equal_raws(df1, raw1, df2):
  raw_cnt_df2 = df2.shape[0]
  col_cnt_df2 = df2.shape[1]
  print(f'Ищем совпадения строки {raw1} первого датафрейма во втором...')
  for i in range(raw_cnt_df2):
#    print(i, sep=' ', end='|')
    if col_cnt_df2==(df1.iloc[raw1,:]==df2.iloc[i,:])[:].to_list().count(True):
      print(f'{i}', sep=' ', end='; ')
  print('\nПоиск совпаающих строк завершён')

## Детальный просмотр содержания столбцов заданного типа

In [ ]:
# Визуализация заполненности столбцов
##
# df - обрабатываемый датафрейм
#  columns_type - тип столбца: ['float64', 'object']
def see_columns(df, columns_type):
  if columns_type not in ['float64', 'obfect']:
    print('!!! Ошибочный параметр в "see_columns(columns_type)..."')
    return
  dfo = df.select_dtypes(include=[columns_type])
  col_cnt = 0
  for colname in dfo.columns:
    col_cnt += 1
#    if (col_cnt==1 or col_cnt==20 or col_cnt==300):
    unique_list = list(dfo.loc[~dfo[colname].isnull(), colname].unique())
    print('^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^')
    print(f'col№:{col_cnt} *** {colname}: ***')
    print('==================================')
    print('Количество непустых элементов: ', len(dfo[~dfo[colname].isna()]))
    print('Количество уникальных элементов: ', len(unique_list), '')
    print('----------------------------------')
    print('....уникальная запись.... = число повторов')
    print('[индексы строк с записями...]')
    # unique_dict = dict.fromkeys(unique_list, -1)
    for unique_name in unique_list:
      df_col_un = dfo[dfo[colname]==unique_name]
      print(f'->{unique_name} = {len(df_col_un)}')
      print(df_col_un.index.to_list())
      print(df.loc[df_col_un.index,ZMK_NM].to_list())

In [ ]:
df1.loc[[11,111],ZMK_NM].to_list()

['Годен', 'Годен']



---



## Сжатие столбцов заданного типа в указанных параметрах

In [ ]:
# Сокращение количества столбцов
##
def shrinking_columns_quantity(df, # обрабатываемый датафрейм
                               columns_type, # тип столбца: ['float64', 'object']
                               # пороговое значение отсечения по уровню заполненности столбца
                               fill_threshold_quantity,
                               # пороговое значение отсечения по количеству уникальных элементов
                               unique_threshold_quantity):
  print('__________shrinking_columns_quantity(...)__________\n')
  if columns_type not in ['float64', 'obfect']:
    print('!!! Ошибочный параметр "columns_type" в функции "shrinking_columns_quantity(...)"')
    return
  #
  print('"shrinking_columns_quantity()": Стартовый размер датасета:', df.shape)
  dff = pd.DataFrame(columns=['col', 'fill', 'uniq', 'col_ind'])
  # dfo = df.select_dtypes(include=['object'])
  dfo = df.select_dtypes(include=['float64'])
  for colname in dfo.columns:
    dff.loc[len(dff.index)] = [
        colname,
        len(dfo[~dfo[colname].isna()]),
        len(dfo.loc[~dfo[colname].isnull(), colname].unique()),
        df.columns.get_loc(colname)
    ]
  #dff.sort_values(
  #        by = ["fill", "uniq"],
  #        ascending = False)
  print('Всего в обработке столбцов: ', len(dff))

  dff_fill = dff[dff['fill']>fill_threshold_quantity]
  print(f'Количество столбцов с заполненностью > {fill_threshold_quantity} =',
        len(dff_fill))
  dff_fill_uniq = dff_fill[dff_fill_50_plus['uniq']>unique_threshold_quantity]
  print(f'Количество из предыдущих с уникальными элементами > {unique_threshold_quantity} (оставляемых столбцов) =',
        len(dff_fill_uniq))
  print(f'Характеристика оставляемых столбцов:\n\
  Заполненность: min={dff_fill_uniq["fill"].min()}; max={dff_fill_uniq["fill"].max()}\n\
  Уникальные:min={dff_fill_uniq["uniq"].min()}; max={dff_fill_uniq["uniq"].max()}\n')

  df.drop(df.columns[dff['col_ind'].index.difference(dff_fill_uniq['col_ind'].index)],
          axis= 1,
          inplace=True)
  print('"shrinking_columns_quantity()": Финальный размер датасета:', df.shape)
  print('__________\n')